In [ ]:
from __future__ import print_function, division
import os
import SimpleITK as sitk
import numpy as np
from glob import glob
import pandas as pd
import cv2
import random
import scipy.ndimage
try:
    from tqdm import tqdm  # long waits are not fun
except:
    print('TQDM does make much nicer wait bars...')
    tqdm = lambda x: x

In [ ]:
subset = "train_subset*/"
# subset = "subset*/"###while need to preprocess luna ,chose this
# subset = "val_subset*/"

tianchi_path = "./tianchi"
# tianchi_path = "./LUNA 2016"###while need to preprocess luna16 ,chose this

output_path = "./tianchi/tianchi-2D/"

In [ ]:
train_data_path = os.path.join(output_path, "traindata/")#subset)

print("train_data_path: %s" % train_data_path)
train_images = glob(train_data_path + "*.npy")
print(train_images)

tmp_workspace = os.path.join(output_path, "64/")###the width we need to cut,it can also be 48,80,96,112...

if not os.path.exists(tmp_workspace):
    os.makedirs(tmp_workspace)

mask_workspace = os.path.join(output_path, "trainlabel/")

In [ ]:
#
# Helper function to get rows in data frame associated
# with each file
def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return (f)


#
# The locations of the nodes
df_node = pd.read_csv(tianchi_path + "/csv/train/annotations.csv")
# df_node = pd.read_csv(tianchi_path + "/CSVFILES/annotations.csv")###while need to preprocess luna ,chose this
# df_node = pd.read_csv(tianchi_path + "/csv/val/annotations.csv")
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(train_images, file_name))
df_node = df_node.dropna()


In [ ]:
for fcount, img_file in enumerate(tqdm(train_images)):
    mini_df = df_node[df_node["file"] == img_file]  # get all nodules associate with file
    if mini_df.shape[0] > 0:  # some files may not have a nodule--skipping those
        img_array = np.load(img_file)
        masks=np.load(img_file.replace('data','label'))
        height, width, num_z =img_array.shape 
        oy,ox,oz=list(map(float,img_file.split('_')[-4:-1] ))
        h=7
        h1=h//2
        r=32 #it is the half of width we need to cut,it can also be 24,40,48,56...
        for node_idx, cur_row in mini_df.iterrows():
            node_x = round(min(max(2*r-2,cur_row["coordX"]-ox),width-2*r+2))
            node_y = round(min(max(2*r-2,cur_row["coordY"]-oy),height-2*r+2))
            node_z = cur_row["coordZ"]-oz
            diam = cur_row["diameter_mm"]
            quater=diam/4
            value=np.arange(max(h1,round(node_z-h1-quater)),min(round(node_z+h1+1+quater),num_z-h1))
            middle=np.random.randint(2-r,r-2,size=[len(value),2])+[node_y,node_x]
            for num,post in enumerate(middle):
                img=img_array[post[0]-r:post[0]+r,post[1]-r:post[1]+r,value[num]-h1:value[num]+h1+1]
                mask=masks[post[0]-r:post[0]+r,post[1]-r:post[1]+r,value[num]-h1:value[num]+h1+1,:]
                np.save(tmp_workspace+'image_%s_%s_%s_%s.npy'%(cur_row["seriesuid"],post[0],post[1],value[num]),img)
                np.save(tmp_workspace+'mask_%s_%s_%s_%s.npy'%(cur_row["seriesuid"],post[0],post[1],value[num]),mask)